In [1]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir("../")
print(os.getcwd())

c:\workspace\EagSession1\eag_agentic_rag\url_rag


In [3]:
from IPython.display import display, HTML, Markdown
from dotenv import load_dotenv

In [4]:
# Load environment variables
load_dotenv()

# Clear SSL_CERT_FILE environment variable if set
if "SSL_CERT_FILE" in os.environ:
    del os.environ["SSL_CERT_FILE"]

In [5]:
from utility.llm_provider import default_llm
from utility.embedding_provider import OpenAIEmbeddingProvider
from utility.utils import read_yaml_file

In [6]:
llm = default_llm.chat_model
embedder = OpenAIEmbeddingProvider().embeddings

In [7]:
print(llm.invoke("what is the capital of France?"))
print(len(embedder.embed_query("what is the capital of France?")))

content='The capital of France is Paris.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90122d973c', 'id': 'chatcmpl-BU9rxe7qNlzRKsFlSXdww3AuRo4Ja', 'finish_reason': 'stop', 'logprobs': None} id='run-2c58d0b5-c640-44ae-8ed8-149a6f02b3ea-0' usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
1536


In [8]:
# read config
config = read_yaml_file("utility/config.yaml")
print(config)

{'db_index_name': 'weburl_index', 'history_index_name': 'history_index', 'chunk_size': 500, 'chunk_overlap': 50, 'model_name': 'gpt-4o', 'reset_index': True}


In [9]:
history_index_name = config["history_index_name"]
history_index_name = os.path.join(os.getcwd(), history_index_name)
print(history_index_name)

c:\workspace\EagSession1\eag_agentic_rag\url_rag\history_index


In [10]:
import uuid
from client.memory import FaissConversationStore

In [11]:
# conv_id = uuid.uuid4()
# store = FaissConversationStore(
#     embedder, index_folder=history_index_name, reset_index=True
# )

2025-05-06 16:01:18.673 | WARNING  | utility.utils:check_and_reset_index:25 - reset_index is set to True. Deleting existing index at 'c:\workspace\EagSession1\eag_agentic_rag\url_rag\history_index'
2025-05-06 16:01:18.676 | SUCCESS  | utility.utils:check_and_reset_index:30 - Successfully deleted index folder 'c:\workspace\EagSession1\eag_agentic_rag\url_rag\history_index'


In [12]:
# messages = [
#     {"sender": "human", "content": "Hello!"},
#     {"sender": "ai", "content": "Hi, how can I help you?"},
#     {"sender": "human", "content": "Tell me a joke."},
#     {"sender": "ai", "content": "Why did the chicken cross the road?"},
# ]
# store.store_conversation(str(conv_id), messages)
# print(store.get_conversation(str(conv_id)))

In [13]:
# # new conversation with same id
# messages = [
#     {"sender": "human", "content": "Tell me a joke about a space alien"},
#     {"sender": "ai", "content": "Why did the space alien cross the road?"},
#     {"sender": "human", "content": "What is the capital of France?"},
#     {"sender": "ai", "content": "The capital of France is Paris."},
# ]
# store.store_conversation(str(conv_id), messages)
# print(store.get_conversation(str(conv_id)))

#### Next steps 
- Perception code which will be used prepare the question based on the user query and chat history
- Decision making code which will be used to decide the action based on the question and chat history
- Action code which will be used to execute the action
- Memory code which will be used to store the chat history
- Client code which will be used to coordinate the conversation between the perception, decision making, action and memory


##### Perception code

In [14]:
from pydantic import BaseModel, Field
from typing import Literal, Optional
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableSequence
import json

In [15]:
class WebContentSearch(BaseModel):
    enhanced_user_query: str = Field(
        description="The enhanced user query to be searched on the web for similar content stored in the memory"
    )
    no_of_results: int = Field(
        1,
        description="The number of results to be returned from the web search corresponding to the user query",
    )

In [16]:
system_prompt = """
You are an intelligent assistant designed to refine user queries using current input and prior context. Your goal is to construct a precise, context-aware enhanced query that reflects the user’s true intent.

Your responsibilities are:
1. Analyze the current user message to understand the core request.
2. Examine the chat history (a list of messages with sender and content) to identify:
   - Any contextually relevant information.
   - Follow-ups or references to previous discussions.
3. Determine whether the current query is:
   a. A continuation of a previous topic.
   b. A new, unrelated query.

Based on this analysis, follow the appropriate path:

**If the current message relates to previous topics:**
- Incorporate relevant prior context into the enhanced query.
- Clarify ambiguous references or pronouns using information from the chat history.
- Resolve under-specified requests by grounding them in previous conversations.

**If the message is a new topic:**
- Focus solely on the current message.
- Do not infer or inject unrelated context from chat history.

**Process for forming the enhanced user query:**
1. Carefully review the current message.
2. Analyze the chat history (most recent last) to extract relevant past content.
3. Identify if the user is building on a previous conversation.
4. Integrate any necessary clarifications, references, or details from the chat history.
5. Output a concise, context-enriched, unambiguous enhanced query.

**Inputs:**
- `chat_history`: A list of dictionaries, each with keys `sender` and `content`.
    {chat_history}
- `user_query`: The latest message from the user.
    {user_query}

**Output Format:**
{format_instructions}
"""


In [17]:
def get_perception_chain(llm) -> RunnableSequence:
    """
    Creates and returns the perception chain for extracting travel search parameters.

    Args:
        default_llm: The default LLM to use for the chain.

    Returns:
        A LangChain runnable sequence that takes a user query and chat history,
        and returns a TravelSearch object.
    """

    # Set up a parser
    parser = PydanticOutputParser(pydantic_object=WebContentSearch)

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "The user query is: {user_query}"),
        ]
    )

    prompt = prompt.partial(format_instructions=parser.get_format_instructions())

    # Create and return the chain
    return prompt | llm | parser

In [27]:
conv_id = uuid.uuid4()
store = FaissConversationStore(
    embedder, index_folder=history_index_name, reset_index=True
)

2025-05-06 16:02:54.829 | WARNING  | utility.utils:check_and_reset_index:25 - reset_index is set to True. Deleting existing index at 'c:\workspace\EagSession1\eag_agentic_rag\url_rag\history_index'
2025-05-06 16:02:54.831 | SUCCESS  | utility.utils:check_and_reset_index:30 - Successfully deleted index folder 'c:\workspace\EagSession1\eag_agentic_rag\url_rag\history_index'


In [28]:
chat_history = store.get_conversation_as_lc_messages(str(conv_id))
print(chat_history)

[]


In [29]:
user_query = "What is role of helm in kubernetes?"

In [30]:
perception_chain = get_perception_chain(llm)

In [32]:
result = perception_chain.invoke(
    {"user_query": user_query, "chat_history": chat_history}
)
print(result)

enhanced_user_query='What is the role of Helm in Kubernetes?' no_of_results=1


In [33]:
# create the history object
messages = [
    {"sender": "human", "content": user_query},
    {"sender": "ai", "content": result.enhanced_user_query},
]
store.store_conversation(str(conv_id), messages)

In [34]:
user_query = "Give me two articles on for the same topic?"
chat_history = store.get_conversation_as_lc_messages(str(conv_id))
print(chat_history)

result = perception_chain.invoke(
    {"user_query": user_query, "chat_history": chat_history}
)
print(result)

[{'role': 'human', 'content': 'What is role of helm in kubernetes?'}, {'role': 'ai', 'content': 'What is the role of Helm in Kubernetes?'}]
enhanced_user_query='Give me two articles on the role of Helm in Kubernetes.' no_of_results=2


In [36]:
# create the history object
messages = [
    {"sender": "human", "content": user_query},
    {"sender": "ai", "content": result.enhanced_user_query},
]
store.store_conversation(str(conv_id), messages)

In [37]:
chat_history = store.get_conversation_as_lc_messages(str(conv_id))
print(chat_history) 

[{'role': 'human', 'content': 'What is role of helm in kubernetes?'}, {'role': 'ai', 'content': 'What is the role of Helm in Kubernetes?'}, {'role': 'human', 'content': 'Give me two articles on for the same topic?'}, {'role': 'ai', 'content': 'Give me two articles on the role of Helm in Kubernetes.'}]


In [38]:
# create a function where we will pass the history object and a list of conversation and the chain
# and it will keep processing the conversation and return the enhanced user query


def process_conversation_and_enhance_query(
    history_store, conversation_list, chain, conv_id
):
    """
    Processes a list of user queries (conversation_list) using the provided chain and history store.
    For each user query, it retrieves the chat history, invokes the chain, stores the conversation,
    and returns a list of enhanced user queries.

    Args:
        history_store: The object responsible for storing and retrieving conversation history.
        conversation_list: List of user queries (strings) to process.
        chain: The chain object with an 'invoke' method.
        conv_id: The conversation id to store the conversation.

    Returns:
        List of enhanced user queries (one for each user query in conversation_list).
    """
    enhanced_queries = []
    for user_query in conversation_list:
        # Retrieve chat history for the conversation
        chat_history = history_store.get_conversation_as_lc_messages(str(conv_id))
        # Invoke the chain to get the result
        result = chain.invoke({"user_query": user_query, "chat_history": chat_history})
        # Store the conversation
        messages = [
            {"sender": "human", "content": user_query},
            {"sender": "ai", "content": result.enhanced_user_query},
        ]
        history_store.store_conversation(str(conv_id), messages)
        # Collect the enhanced user query
        enhanced_queries.append(result.enhanced_user_query)
    return enhanced_queries

In [39]:
conv_id = uuid.uuid4()
memory_store = FaissConversationStore(
    embedder, index_folder=history_index_name, reset_index=True
)
conversation_list = [
    "What is the capital of France?",
    "Tell me more about its history.",
    "List two famous landmarks in that city."
]
enhanced_queries = process_conversation_and_enhance_query(memory_store, conversation_list, perception_chain, conv_id)
print(enhanced_queries)


2025-05-06 16:07:17.994 | WARNING  | utility.utils:check_and_reset_index:25 - reset_index is set to True. Deleting existing index at 'c:\workspace\EagSession1\eag_agentic_rag\url_rag\history_index'
2025-05-06 16:07:17.995 | SUCCESS  | utility.utils:check_and_reset_index:30 - Successfully deleted index folder 'c:\workspace\EagSession1\eag_agentic_rag\url_rag\history_index'


['What is the capital city of France?', 'Tell me more about the history of Paris, the capital of France.', 'List two famous landmarks in Paris, the capital of France.']


######################################### END OF FILE #########################################